In [1]:
import os
import re
import json

from typing import Dict, List

def load_jsonl(file):
    with open(file, "r") as f:
        data = []
        for line in f:
            data.append(json.loads(line))
    return data

In [28]:
# file_path = r"../infer/output/generated_predictions_qwen_0.jsonl"

# data = load_jsonl(file_path)

In [2]:
prompt_template = """
你是资深的产业专家！请先浏览下述产业环节的描述文本。
{name} {node}产业链环节：{desc}
接下来，请你逐一浏览行业分类列表：{selected_elements}，并在其中找出一眼看上去就属于上述产业链环节的行业，不需要做过渡的行业内容挖掘。
以python列表的格式输出，若没有密切相关的行业就返回空列表[]。输出样例：["xxx", "xxx"]
""".strip()

In [3]:
def parse_item(data) -> Dict[str, List]:
    industry_nodes = []
    for item in data:
        industry_node = re.search(r"\n(.*?)产业链环节", item["prompt"])
        if industry_node:
            industry_node = industry_node.group(1)
        pred = re.search("(\[.*?\])", item["predict"], re.DOTALL)
        no_pred = 0
        
        if pred:
            pred = pred.group(1)
            try:
                pred = eval(pred)
                if len(pred) > 0 and isinstance(pred[0], str):
                    pred = pred
                else:
                    pred = []
            except:
                pred = []
        
        else:
            no_pred += 1
            pred = []
        industry_nodes.append([industry_node, pred])

    # 次数统计
    preds = {}
    for k, pred in industry_nodes:
        # try:
        #     pred = eval(pred)
        # except:
        #     pred = []

        if k not in preds.keys():
            preds[k] = []
        
        preds[k].extend(pred)

    return preds

In [32]:
# parse_item(data)

In [4]:
ans = {}
for file in os.listdir(p := "../infer/output/qwen"):
    if not file.endswith(".jsonl"):
        continue
    
    file_name = os.path.join(p, file)
    print(file_name)
    data = load_jsonl(file_name)

    print(data[0])


../infer/output/qwen/generated_predictions_qwen_0.jsonl
{'prompt': '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n你是资深的产业专家！请先浏览下述产业环节。\n集成电路 芯片设计产业链环节。\n接下来，请你逐一浏览行业分类列表：[\'公证服务\', \'计算机及通讯设备经营租赁\', \'摩托车及零配件批发\', \'喷枪及类似器具制造\', \'金属制品修理\']，并在其中找出一眼看上去就属于集成电路 芯片设计产业链环节的行业，不需要做过渡的行业内容挖掘。\n以python列表的格式输出，若没有密切相关的行业就返回空列表[]。输出样例：["xxx", "xxx"]<|im_end|>\n<|im_start|>assistant\n', 'predict': '根据提供的行业分类列表，没有一眼看上去就属于集成电路 芯片设计产业链环节的行业。因此返回空列表：\n\n[]', 'label': '<|im_end|>\n'}
../infer/output/qwen/generated_predictions_qwen_1.jsonl
{'prompt': '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n你是资深的产业专家！请先浏览下述产业环节。\n集成电路 芯片设计产业链环节。\n接下来，请你逐一浏览行业分类列表：[\'汽车车身、挂车制造\', \'天然水收集与分配\', \'其他原动设备制造\', \'汽车金融公司服务\', \'石油和天然气开采专业及辅助性活动\']，并在其中找出一眼看上去就属于集成电路 芯片设计产业链环节的行业，不需要做过渡的行业内容挖掘。\n以python列表的格式输出，若没有密切相关的行业就返回空列表[]。输出样例：["xxx", "xxx"]<|im_end|>\n<|im_start|>assistant\n', 'predict': '根据提供的行业分类列表，没有一眼看上去就属于集成电路 芯片设计产业链环节的行业。因此返回空列表：\n\n[]', 'l

In [5]:
ans = {}
for file in os.listdir(p := "../infer/output/qwen"):
    if not file.endswith(".jsonl"):
        continue
    
    file_name = os.path.join(p, file)
    print(file_name)
    data = load_jsonl(file_name)
    preds = parse_item(data)

    for k, industry_cls in preds.items():
        if k not in ans.keys():
            ans[k] = {}
            
        for item in industry_cls:
            if item not in ans[k].keys():
                ans[k][item] = 0
            ans[k][item] += 1

../infer/output/qwen/generated_predictions_qwen_0.jsonl
../infer/output/qwen/generated_predictions_qwen_1.jsonl
../infer/output/qwen/generated_predictions_qwen_2.jsonl
../infer/output/qwen/generated_predictions_qwen_3.jsonl
../infer/output/qwen/generated_predictions_qwen_4.jsonl
../infer/output/qwen/generated_predictions_qwen_5.jsonl
../infer/output/qwen/generated_predictions_qwen_6.jsonl
../infer/output/qwen/generated_predictions_qwen_7.jsonl


In [6]:
ans

{'集成电路 芯片设计': {'光电子器件制造': 1,
  '集成电路设计': 8,
  '集成电路制造': 7,
  '半导体器件专用设备制造': 6,
  '半导体分立器件制造': 7,
  '电子电路制造': 4,
  '专业设计服务': 2,
  '可穿戴智能设备制造': 1,
  '服务消费机器人制造': 1,
  '电子元器件与机电组件设备制造': 1,
  '电阻电容电感元件制造': 2,
  '电子专用材料制造': 1,
  '电气信号设备装置制造': 1},
 '集成电路 芯片制造': {'可穿戴智能设备制造': 1,
  '电容器及其配套设备制造': 1,
  '电子专用材料制造': 7,
  '集成电路设计': 8,
  '半导体器件专用设备制造': 8,
  '电子元器件与机电组件设备制造': 4,
  '集成电路制造': 8,
  '其他专用仪器制造': 1,
  '半导体分立器件制造': 8,
  '电子电路制造': 7,
  '电子真空器件制造': 2,
  '电子测量仪器制造': 2,
  '电阻电容电感元件制造': 3,
  '计算机零部件制造': 1,
  '无机碱制造': 1,
  '本册印制': 1,
  '基础软件开发': 1,
  '化学试剂和助剂制造': 1,
  '机械零部件加工': 1,
  '电力电子元器件制造': 2,
  '硅冶炼': 1,
  '气体、液体分离及纯净设备制造': 1,
  '专业设计服务': 1,
  '其他电子设备制造': 1,
  '工程和技术研究和试验发展': 1},
 '集成电路 应用与系统集成': {'集成电路设计': 8,
  '通信系统设备制造': 4,
  '集成电路制造': 7,
  '其他智能消费设备制造': 1,
  '电子元器件与机电组件设备制造': 3,
  '信息处理和存储支持服务': 2,
  '显示器件制造': 1,
  '电力电子元器件制造': 3,
  '汽车零部件及配件制造': 2,
  '运输设备及生产用计数仪表制造': 1,
  '工业控制计算机及系统制造': 7,
  '光电子器件制造': 1,
  '计算机整机制造': 6,
  '计算机零部件制造': 4,
  '可穿戴智能设备制造': 2,
  '信息系统集成服务': 6,
  '计算机及通讯

In [7]:
ans.keys()

dict_keys(['集成电路 芯片设计', '集成电路 芯片制造', '集成电路 应用与系统集成', '网络安全 安全技术研发', '网络安全 安全产品开发', '网络安全 安全服务提供', '电力装备 电力设备研发与设计', '电力装备 电力设备制造与组装', '电力装备 电力设施建设与运维', '安全应急装备 安全应急技术研发', '安全应急装备 安全应急装备制造', '安全应急装备 应急服务与救援应用'])

In [8]:
for k, industry_cls in preds.items():
    print(k, industry_cls)

集成电路 芯片设计 ['电子电路制造', '电气信号设备装置制造', '电阻电容电感元件制造', '集成电路制造', '半导体器件专用设备制造', '集成电路设计', '半导体分立器件制造']
集成电路 芯片制造 ['电力电子元器件制造', '电子电路制造', '集成电路制造', '集成电路设计', '半导体器件专用设备制造', '电子专用材料制造', '专业设计服务', '其他电子设备制造', '半导体分立器件制造', '工程和技术研究和试验发展']
集成电路 应用与系统集成 ['医疗诊断、监护及治疗设备制造', '工业自动控制系统装置制造', '电气信号设备装置制造', '雷达及配套设备制造', '计算机整机制造', '可穿戴智能设备制造', '汽车零部件及配件制造', '智能无人飞行器制造', '物联网技术服务', '集成电路制造', '工业控制计算机及系统制造', '信息系统集成服务', '信息处理和存储支持服务', '电力电子元器件制造', '集成电路设计']
网络安全 安全技术研发 ['安全系统监控服务', '信息安全设备制造', '互联网安全服务']
网络安全 安全产品开发 ['互联网安全服务', '信息技术咨询服务', '信息系统集成服务', '信息安全设备制造', '社会公共安全设备及器材制造']
网络安全 安全服务提供 ['公共安全管理机构', '安全服务', '互联网其他信息服务', '信息安全设备制造', '安全系统监控服务', '社会公共安全设备及器材制造', '互联网安全服务']
电力装备 电力设备研发与设计 ['水轮机及辅机制造', '发电机及发电机组制造', '电力电子元器件制造', '其他输配电及控制设备制造', '电线、电缆制造', '半导体分立器件制造', '变压器、整流器和电感器制造', '电阻电容电感元件制造', '配电开关控制设备制造', '电动机制造', '核电工程施工', '绝缘制品制造', '敏感元件及传感器制造', '滑动轴承制造', '金属切削机床制造', '电容器及其配套设备制造']
电力装备 电力设备制造与组装 ['绝缘制品制造', '水轮机及辅机制造', '发电机及发电机组制造', '电阻电容电感元件制造', '电动机制造', '锅炉及辅助设备制造', '电工仪器仪表制造', '其他电工器材制造', '汽轮

## 输出

In [9]:
import pandas as pd

INDUSTRY_NAME = "产业链名"
INDUSTRY_NODE = "产业链环节"
INDUSTRY_DESC = "产业链环节描述"

df = pd.read_excel("../六链五群行业分类.xlsx")
df = df.rename(columns={"六链": INDUSTRY_NAME})

industry_cls_df = pd.read_excel("../小类注释_含一二三位数代码及名称.xlsx")
industry_cls_df = industry_cls_df[["小类代码2017", "小类名称2017"]]
industry_cls_df = industry_cls_df.rename(
    columns={"小类代码2017": "code", "小类名称2017": "name"}
)
industry_cls_df["code"] = industry_cls_df["code"].map(lambda x: str(x).zfill(4))
ALL_CLS_NAME = industry_cls_df["name"].values.tolist()

In [10]:
output_fold = "output/qwen_14B"
if not os.path.exists(output_fold):
    os.makedirs(output_fold, exist_ok=True)

for k, value in ans.items():
    del_cnt = 0
    for kw, cnt in value.items():
        if kw not in ALL_CLS_NAME:
            del value[kw]
            del_cnt += 1
    tmp_data = []
    for kw, cnt in value.items():
        tmp_data.append([kw, cnt])
    tmp_data.sort(key=lambda x: x[-1], reverse=True)

    tmp_df = pd.DataFrame(tmp_data, columns=["小类行业名", "投票数"])
    tmp_df.to_csv(os.path.join(output_fold, f"{k}.csv"), index=False)
    if del_cnt > 0:
        print(del_cnt, "个重要行业由于不在制定类别中，被移除")